In [1]:
# always check your python version
!python --version

Python 3.11.9


In [2]:
# AWS SDK for Python, -q flag is for quiet
%pip install -q boto3

Note: you may need to restart the kernel to use updated packages.


In [3]:
import boto3
bedrock_client = boto3.client('bedrock-runtime')

In [4]:
import random
from datetime import datetime, timedelta, timezone
from tools import get_date_time, get_weather, get_historical_fact, weather_tools
from helper import load_text_file

In [18]:
# https://gist.github.com/labeveryday/1ce04d4169a638a4e267384100affad5
class ToolAgent:
  def __init__(self):
    self.client = boto3.client('bedrock-runtime')
    self.messages = [] # load with initial messages  
    self.model_id = 'anthropic.claude-3-5-sonnet-20240620-v1:0'
  def chat(self,text):
    # Observersation
    resp = self.converse(text)
    self.messages.append(resp)
    # Action
    messages = self.process_action(resp)
    return messages # [TODO]
  def converse(self,text=''):
    system_prompt_text = load_text_file("prompts/weather.txt")    
    system_prompt = [{"text": system_prompt_text}]      
    self.messages.append({"role": "user", "content": [{"text": text}]})
    # https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/converse.html
    resp = self.client.converse(
      modelId=self.model_id,
      messages=self.messages,
      system=system_prompt,
      inferenceConfig={
        "temperature": 0.5
      },        
      toolConfig={
        "tools": weather_tools,
        "toolChoice": {"auto": {}}
      }
    )
    return resp['output']['message']
  def process_action(self,ai_response):
    print("process_action:ai_response")
    print(ai_response)
    for content in ai_response['content']:
      if 'text' in content:
        return f"AI: {content['text']}"
      elif 'toolUse' in content:
        tool_use = content['toolUse']
        tool_name = tool_use['name']
        return f"# Using the {tool_use['name']} tool..."
        
        tool_functions = {
          'get_date_time': get_date_time,
          'get_weather': lambda: get_weather(tool_use['input']['location']),
          'get_historical_fact': get_historical_fact
        }
          
        if tool_name in tool_functions:
          try:
            tool_result = tool_functions[tool_name]()
            self.messages.append({
              "role": "user",
              "content": [{
                "toolResult": {
                  "toolUseId": tool_use['toolUseId'],
                  "content": [{"json": tool_result}],
                  "status": "success"
                }
              }]
            })
            result = self.converse()
            self.messages.append(result)
            return f"AI: {result['content'][0]['text']}"
          except Exception as e:
            return f"Error using {tool_name} tool: {str(e)}"


In [11]:
agent = ToolAgent()
print(agent.chat("What is the weather today in Toronto Ontario Canada?")) # weather tool

process_action:ai_response
{'role': 'assistant', 'content': [{'text': "Certainly! I'd be happy to check the current weather for Toronto, Ontario, Canada. Let me use the weather tool to get that information for you."}, {'toolUse': {'toolUseId': 'tooluse_kJKeXC27QDCu9pTdDuz3WA', 'name': 'get_weather', 'input': {'location': 'Toronto, Ontario, Canada'}}}]}
AI: Certainly! I'd be happy to check the current weather for Toronto, Ontario, Canada. Let me use the weather tool to get that information for you.


In [13]:
print(agent.chat("What is the weather today in Toronto Ontario Canada?")) # it should error out eg. TooManyRequest or You need to further the conversation

ValidationException: An error occurred (ValidationException) when calling the Converse operation: A conversation must alternate between user and assistant roles. Make sure the conversation alternates between user and assistant roles and try again.

In [20]:
agent = ToolAgent()
# Update the facts with a fact and date with what happen on this day.
print(agent.chat("What historical thing happend on this date?")) 

process_action:ai_response
{'role': 'assistant', 'content': [{'text': "To answer your question about what historical event happened on this date, I'll need to use the get_historical_fact tool. This tool provides information about a significant historical event that occurred on the current date. Let me fetch that information for you."}, {'toolUse': {'toolUseId': 'tooluse_iiUOORPxQBKrNHwAH3TrzQ', 'name': 'get_historical_fact', 'input': {}}}]}
AI: To answer your question about what historical event happened on this date, I'll need to use the get_historical_fact tool. This tool provides information about a significant historical event that occurred on the current date. Let me fetch that information for you.


In [23]:
agent = ToolAgent()
print(agent.chat("What is the time?")) 

ThrottlingException: An error occurred (ThrottlingException) when calling the Converse operation (reached max retries: 4): Too many requests, please wait before trying again. You have sent too many requests.  Wait before trying again.